In [ ]:
%run ./util

In [ ]:
import json
from datetime import datetime, timedelta
import requests
import time

ParDict=json.loads(dbutils.widgets.get('NBParameters'))
DBToken=dbutils.widgets.get('DBToken')
JobId=dbutils.widgets.get('JobId')
workspaceId=dbutils.widgets.get('workspaceId')

In [ ]:
def GetLastExecution(urlJobs, DBToken, JobId, activeState):
        payload="{\r\n      \"path\": \"/\"\r\n      \r\n}"
        headers = {
            'Authorization': 'Bearer {0}'.format(DBToken),
            'Content-Type': 'text/plain'
        }
        #extracts last execution
        if activeState == True:
          activeStr="true"
        else:
          activeStr="false"
        url = "{0}/runs/list?&job_id={1}&active_only={2}&limit=1".format(urlJobs, JobId,activeStr)
        response = requests.request("GET", url, headers=headers, data=payload)
        return json.loads(response.text)

      
def CancelJobExecution(urlJobs, DBToken,JobId, run_id, waitForCancellation, CancellationTimeS, SleepWait):
        urlCancel="{0}/runs/cancel".format(urlJobs)
        headers = {
            'Authorization': 'Bearer {0}'.format(DBToken),
            'Content-Type': 'text/plain'
        }
        payload={ "run_id": run_id}
        response = requests.request("POST", urlCancel, headers=headers, data=json.dumps(payload))
        dateTMP=datetime.now()
        while  (dateTMP + timedelta(seconds=CancellationTimeS) > datetime.now()) and waitForCancellation == True:
          time.sleep(SleepWait)
          ResJobStatus=GetLastExecution(urlJobs, DBToken, JobId, False)
          Status=ResJobStatus["runs"][0]["state"]["life_cycle_state"]
          if  Status not in str(["RUNNING", "PENDING","TERMINATING"]):
            break
        
        #print(ResJobStatus)
        if waitForCancellation == True and ResJobStatus["runs"][0]["state"]["result_state"] != "CANCELED":
          print("{0} ERROR: Not possilbe to cancel job {1} , Run {2} !!".format(GETDATE(), JobId, run_id))
          a=1/0

def TriggersExecution(urlJobs, DBToken, TriggerTimeS, JobId):
        urlTrgigger="{0}/run-now".format(urlJobs)
        headers = {
            'Authorization': 'Bearer {0}'.format(DBToken),
            'Content-Type': 'text/plain'
        }
        payload={ "job_id": JobId}
        response = requests.request("POST", urlTrgigger, headers=headers, data=json.dumps(payload))
        if response.status_code == 200:
              LogMsg="{0}:Starting job {1}...".format(GETDATE(),JobId)
              print(LogMsg)
              time.sleep(TriggerTimeS)
              return GetLastExecution(urlJobs, DBToken,  JobId, True)
        else:
          print(response.text)
        return None
      
def ExecuteDatabricksJob(ProcessConfig, JobId,DBToken, workspaceId):
        # Starts the Refresh data set Job
        SleepWait=ProcessConfig["PausetimeS"]
        ResultTimeS=ProcessConfig["ResultTimeS"]
        TriggerTimeS=ProcessConfig["TriggerTimeS"]
        WaitingTimeS=ProcessConfig["WaitingTimeS"]
        CancellationTimeS=ProcessConfig["CancellationTimeS"]
        dateTMP=datetime.now()
 
        urlJobs = "https://{0}/api/2.0/jobs".format(workspaceId)
        # Checks if the job is being executed
        ResJobStatus=GetLastExecution(urlJobs, DBToken, JobId, True)
        if "runs" in ResJobStatus:
          # There is already an execution, must be cancelled
          run_id=ResJobStatus["runs"][0]["run_id"]
          CancelJobExecution(urlJobs, DBToken,JobId, run_id, True, CancellationTimeS, SleepWait)
        
        ResJobStatus=GetLastExecution(urlJobs, DBToken, JobId, True)
        if "runs" not in ResJobStatus:
          #Triggers execution
          ResJobStatus=TriggersExecution(urlJobs, DBToken, TriggerTimeS, JobId)
         
        if "runs" in ResJobStatus:
            Status=ResJobStatus["runs"][0]["state"]["life_cycle_state"]
            run_id=ResJobStatus["runs"][0]["run_id"]
            # Waits for the job to end
            
            while Status in str(["RUNNING", "PENDING","TERMINATING"]) and (dateTMP + timedelta(seconds=WaitingTimeS) > datetime.now()):
              LogMsg="{0}:Waiting to finish job {1} execution...".format(GETDATE(),JobId)
              print(LogMsg)
              time.sleep(SleepWait)
              ResJobStatus=GetLastExecution(urlJobs, DBToken, JobId, False)
              Status=ResJobStatus["runs"][0]["state"]["life_cycle_state"]
        
            if  (dateTMP + timedelta(seconds=WaitingTimeS) < datetime.now()): # Timeout
              # Cancel job
              CancelJobExecution(urlJobs, DBToken, run_id, False, CancellationTimeS, SleepWait)
              LogMsg="{0}:ERROR Timeout, cancelling job {1}!".format(GETDATE(),JobId)
              a=1/0 # force exception
            else:
              dateTMP=datetime.now()
              while  (dateTMP + timedelta(seconds=ResultTimeS) > datetime.now()):
          
                ResJobStatus=GetLastExecution(urlJobs, DBToken, JobId, False)
                Status=ResJobStatus["runs"][0]["state"]["life_cycle_state"]
                # Failure
                if ResJobStatus["runs"][0]["state"]["life_cycle_state"] == "TERMINATED" and \
                    ResJobStatus["runs"][0]["state"]["result_state"] == "FAILED":
                    LogMsg="{0}:ERROR: Not possible to execute JOB {1} {2}".format(GETDATE(),JobId, response.text)
                    print(LogMsg)
                    a=1/0
                    return -1
                # Success
                if ResJobStatus["runs"][0]["state"]["life_cycle_state"] == "TERMINATED" and \
                    ResJobStatus["runs"][0]["state"]["result_state"] == "SUCCESS":
                    LogMsg="{0}: JOB {1}  execution completed with SUCCESS!!".format(GETDATE(),JobId)
                    print(LogMsg)
                    return 1

                # and waits if has to
                time.sleep(SleepWait)
                LogMsg="{0}:ERROR: Not possible to deliver execution result for JOB {1} !!!".format(GETDATE(),JobId)
                print(LogMsg)
                a=1/0 
      
        else:
          print("ERROR: Not possible to execute job {0}!".format(JobId))
          

In [ ]:
# Main
ExecuteDatabricksJob(ParDict,JobId,DBToken, workspaceId)